## Traffic Violations
Metadata Updated: August 9, 2018

This dataset contains traffic violation information from all electronic traffic violations issued in the County. Any information that can be used to uniquely identify the vehicle, the vehicle owner or the officer issuing the violation will not be published.

Update Frequency: Daily

The whole dataset can be downloaded from https://catalog.data.gov/dataset/traffic-violations-56dda

In [2]:
import pandas as pd

# import  holoviews as hv
# from holoviews import opts
# hv.extension('bokeh')

import numpy as np

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.transform import factor_cmap
from bokeh.models import DatetimeTickFormatter
from bokeh.models.tools import HoverTool
from datetime import datetime as dt

In [3]:
# This is needed in order to reveal the plots in the notebook

output_notebook()

Loading BokehJS ...

In [6]:
df = pd.read_csv('/v/courses/dataexpviz.public/Datasets/D-Interactive_Visualization/traffic-violation-small.csv', parse_dates=['Date Of Stop','Time Of Stop'])
    
df.head().T

,0,1,2,3,4
Date Of Stop,2013-09-24 00:00:00,2017-08-29 00:00:00,2014-12-01 00:00:00,2017-08-29 00:00:00,2017-08-28 00:00:00
Time Of Stop,2022-03-23 17:11:00,2022-03-23 10:19:00,2022-03-23 12:52:00,2022-03-23 09:22:00,2022-03-23 23:41:00
Agency,MCP,MCP,MCP,MCP,MCP
SubAgency,"3rd district, Silver Spring","2nd district, Bethesda","6th district, Gaithersburg / Montgomery Village","3rd district, Silver Spring","6th district, Gaithersburg / Montgomery Village"
Description,DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,DRIVER FAILURE TO OBEY PROPERLY PLACED TRAFFIC...,FAILURE STOP AND YIELD AT THRU HWY,FAILURE YIELD RIGHT OF WAY ON U TURN,FAILURE OF DR. TO MAKE LANE CHANGE TO AVAIL. L...
Location,8804 FLOWER AVE,WISCONSIN AVE@ ELM ST,CHRISTOPHER AVE/MONTGOMERY VILLAGE AVE,CHERRY HILL RD./CALVERTON BLVD.,355 @ SOUTH WESTLAND DRIVE
Latitude,NaN,38.981725,39.162888,39.056975,NaN
Longitude,NaN,-77.092757,-77.229088,-76.954633,NaN
Accident,No,No,No,No,No
Belts,No,No,No,No,No


In [7]:
# Select traffic violations by gender

# let's create a new pandas dataframe
new_df = pd.DataFrame() 

# add the gender column
new_df['gender'] = df.Gender

# add time as it's index and rename it
new_df.index = df['Time Of Stop'].astype('datetime64[ns]')
new_df.index.rename('time', inplace=True)

In [8]:
# to get the hourly rates/counts, we need to resample our data with an hourly interval and sum it
new_df = new_df.resample('h').sum()

# and count the "M"-s and "F"-s as strings
new_df['Male'] = new_df.gender.apply(lambda x: x.count('M'))
new_df['Female'] = new_df.gender.apply(lambda x: x.count('F'))
new_df['Total'] = new_df.gender.apply(lambda x: x.count(''))
                                              

# Use bokeh type for storing the dataframe
source = ColumnDataSource(new_df)

## Plot number of violations by male only

In [9]:
# Plot it as a curve
fig = figure(plot_width=850, plot_height=250, # set dimensions of the plot 
             tools='pan,wheel_zoom,box_select,reset', # add interactivity tools
             x_axis_type='datetime', # Set the x-axis to datetime type
             
             # Add the necessary informations to the plot
             title="Number of traffic violations committed by male drivers ")  # add title

# add a curve plot form the data stored now in 'source'
fig.line('time', 'Male',source=source)


# We can add info later as well
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = '# violations'


show(fig)

## Plot number of violations by both male/female drivers

* add legend
* add hover tool

In [10]:
# Plot it as a curve
fig = figure(plot_width=850, plot_height=250, # set dimensions of the plot 
             tools='pan,wheel_zoom,box_select,reset, hover', # add interactivity tools
             x_axis_type='datetime', # Set the x-axis to datetime type
             
             # Add the necessary informations to the plot
             title="Number of traffic violations committed by drivers (not normalized)")  # add title

# Now that we have multiple lines we will need legend as well
# add a curve plot form the data stored now in 'source'
fig.line('time', 'Male',source=source, line_width=3, legend_label="M")
# add another line
fig.line('time', 'Female',source=source, line_width=3, line_color='red', legend_label="F", )

# We can add info later as well
fig.xaxis.axis_label = 'Date'
fig.yaxis.axis_label = '# violations'


show(fig)